In [132]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import scale
from sklearn.metrics import roc_auc_score

In [124]:
train_fname = '../data/train.csv'
test_fname = '../data/test.csv'
df = pd.read_csv(train_fname, sep=';')
df_test = pd.read_csv(test_fname, sep=';')
(n_samples,n_variables) = df.shape
f_con = ['APP_NB','APP_NB_PAYS','APP_NB_TYPE','NB_CLASSES','NB_ROOT_CLASSES','NB_SECTORS','NB_FIELDS','INV_NB',
        'INV_NB_PAYS','INV_NB_TYPE','cited_n','cited_age_min','cited_age_median','cited_age_max','cited_age_mean',
        'cited_age_std','NB_BACKWARD_NPL','NB_BACKWARD_XY','NB_BACKWARD_I','NB_BACKWARD_AUTRE','NB_BACKWARD_PL',
        'NB_BACKWARD','pct_NB_IPC','pct_NB_IPC_LY','oecd_NB_ROOT_CLASSES','oecd_NB_BACKWARD_PL','oecd_NB_BACKWARD_NPL',
        'IDX_ORIGIN','IDX_RADIC']

In [125]:
df = df.reindex(np.random.permutation(df.index));
X_train = df[f_con].values
y_train = df.VARIABLE_CIBLE == 'GRANTED'
X_test = df_test[f_con].values
imputer = Imputer()
# Imputer permet de combler les trous quand des données manquent. Par défaut il prend la moyenne de la dernière donnée vue
# et de la prochaine. Ces données ne sont donc pas normalisées.
X_train = imputer.fit_transform(X_train)
X_test = imputer.fit_transform(X_test)
# Normalisation des features.
scale(X_train,copy=False);
scale(X_test,copy=False);

In [130]:
taken_samples = n_samples
logreg = LogisticRegression()
logreg.fit_transform(X_train[0:taken_samples],y_train[0:taken_samples]);

In [131]:
y_pred_train = logreg.predict_proba(X_train)[:, 1]
print('Score (optimiste) sur le train : %s' % roc_auc_score(y_train, y_pred_train))

Score (optimiste) sur le train : 0.591343414477


In [91]:
# Application au test set et sauvegarde de la soumission correspondante.
y_pred = logreg.predict_proba(X_test)[:, 1]
np.savetxt('../subs/continue_logreg.txt', y_pred, fmt='%s')

In [ ]:
# Utilisation de la cross-validation pour choisir l'optimisation et le coefficient de régularisation inverse C.
from sklearn.grid_search import GridSearchCV
nb_thread = 4
taken_samples = n_samples
logreg = LogisticRegression()
parameters = {'C':[0.0001,0.001,0.01, 0.1,1., 10., 50., 100., 500., 1000.],'solver':['liblinear','newton-cg','lbfgs']}
def scorer(estimator, X, y):
    return roc_auc_score(y, estimator.predict_proba(X)[:,1])
clf = GridSearchCV(logreg, parameters, scorer, n_jobs = nb_thread)
clf.fit(X_train[0:taken_samples],y_train[0:taken_samples])

In [116]:
C = clf.best_estimator_.C
solver = clf.best_estimator_.solver
print(C)
print(solver)
taken_samples = n_samples
logreg = LogisticRegression(C=C,solver=solver)
logreg.fit_transform(X_train[0:taken_samples],y_train[0:taken_samples]);

0.1
liblinear


In [117]:
y_pred_train = logreg.predict_proba(X_train)[:, 1]
print('Score (optimiste) sur le train : %s' % roc_auc_score(y_train, y_pred_train))

Score (optimiste) sur le train : 0.591341662854


### Conclusion : Le score avec cross-validation est très légérement inférieur au score sans, ce qui est probablement dü à des erreurs numériques. On voit aussi que la CV a choisit presque les mêmes paramètres que les paramètres par défaut... Ce score n'étant pas satisfaisant, il faut se mettre à la recherche d'autre chose !